In [ ]:
# ============================================================================
# DIABETIC RETINOPATHY DETECTION PIPELINE - IET CODEFEST 2025
# ============================================================================
# Complete ML Pipeline using PyTorch and ResNet50
#
# Pipeline Overview:
# 1. Dataset Understanding & Label Cleaning
# 2. Exploratory Data Analysis (EDA)
# 3. Preprocessing & Augmentation
# 4. Model Training (Two-phase ResNet50)
# 5. Explainability (Grad-CAM)
# 6. Model Export (ONNX for Next.js)
# 7. Comprehensive Evaluation
# ============================================================================

print("🚀 Starting Diabetic Retinopathy Detection Pipeline")
print("📋 IET Codefest 2025 - Complete ML Solution")
print("⚡ Optimized for speed and efficiency")

In [ ]:
# ============================================================================
# PACKAGE INSTALLATION
# ============================================================================
# Install all required packages for the pipeline

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install timm albumentations opencv-python-headless
!pip install pytorch-grad-cam onnx onnxruntime scikit-learn
!pip install matplotlib seaborn plotly pandas numpy
!pip install efficientnet-pytorch

print("✅ All packages installed successfully!")

In [ ]:
# ============================================================================
# IMPORTS AND SETUP
# ============================================================================
# Import all necessary libraries for the complete pipeline

import os
import json
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
import re

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
from torchvision import models
import timm

# Image processing imports
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ML utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, f1_score, accuracy_score
)
from sklearn.utils.class_weight import compute_class_weight

# Explainability imports
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# ONNX export imports
import onnx
import onnxruntime as ort

# Configure environment
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

print("✅ All imports loaded successfully!")

In [ ]:
# ============================================================================
# CONFIGURATION SETTINGS
# ============================================================================
# Main configuration for the entire pipeline
# Update DATA_PATH and LABELS_FILE to match your dataset location

CONFIG = {
    # Data paths - UPDATE THESE FOR YOUR DATASET
    'DATA_PATH': '/kaggle/input',          # Update this path to your dataset location
    'LABELS_FILE': 'labels.csv',           # Update this to your labels file name
    
    # Model parameters
    'IMAGE_SIZE': 224,                     # Input image size (224x224)
    'BATCH_SIZE': 32,                      # Training batch size
    'NUM_EPOCHS': 50,                      # Maximum training epochs
    'LEARNING_RATE': 1e-4,                 # Initial learning rate
    'WEIGHT_DECAY': 1e-5,                  # Weight decay for regularization
    'NUM_CLASSES': 5,                      # Number of classification classes
    'MODEL_NAME': 'resnet50',              # Model architecture
    
    # Training parameters
    'PATIENCE': 10,                        # Early stopping patience
    'MIN_DELTA': 0.001,                    # Minimum improvement for early stopping
    'SAVE_PATH': './models/',              # Model save directory
    'RANDOM_SEED': 42                      # Random seed for reproducibility
}

# Create necessary directories
os.makedirs(CONFIG['SAVE_PATH'], exist_ok=True)
os.makedirs('./outputs', exist_ok=True)

# Display configuration
print("⚙️  Pipeline Configuration:")
print("=" * 50)
for key, value in CONFIG.items():
    print(f"{key:<20}: {value}")
print("=" * 50)
print("✅ Configuration loaded successfully!")

In [ ]:
# ============================================================================
# LABEL CLEANING AND NORMALIZATION
# ============================================================================
# Clean and normalize inconsistent labels into 5 standardized classes:
# 0 = No_DR, 1 = Mild, 2 = Moderate, 3 = Severe, 4 = Proliferative_DR

def clean_labels(df):
    """
    Clean and normalize inconsistent labels into 5 standardized classes
    Handles numeric, text, mixed case, extra spaces, and leading zeros
    """
    print("🧹 Starting label cleaning process...")
    
    # Create a copy to avoid modifying original
    df_clean = df.copy()
    
    # Convert to string and strip whitespace
    df_clean['label'] = df_clean['label'].astype(str).str.strip()
    
    # Define comprehensive mapping for various label formats
    label_mapping = {
        # Numeric labels (including leading zeros and decimals)
        '0': 0, '00': 0, '0.0': 0,
        '1': 1, '01': 1, '1.0': 1,
        '2': 2, '02': 2, '2.0': 2,
        '3': 3, '03': 3, '3.0': 3,
        '4': 4, '04': 4, '4.0': 4,
        
        # Text labels (case insensitive variations)
        'NO_DR': 0, 'No_DR': 0, 'no_dr': 0, 'No DR': 0, 'no dr': 0, 'NO DR': 0,
        'MILD': 1, 'Mild': 1, 'mild': 1,
        'MODERATE': 2, 'Moderate': 2, 'moderate': 2,
        'SEVERE': 3, 'Severe': 3, 'severe': 3,
        'PROLIFERATIVE_DR': 4, 'Proliferative_DR': 4, 'proliferative_dr': 4,
        'PROLIFERATIVE DR': 4, 'Proliferative DR': 4, 'proliferative dr': 4
    }
    
    # Apply mapping
    df_clean['label_clean'] = df_clean['label'].map(label_mapping)
    
    # Identify invalid labels
    invalid_mask = df_clean['label_clean'].isna()
    invalid_labels = df_clean[invalid_mask]['label'].unique()
    
    print(f"📊 Found {invalid_mask.sum()} invalid labels: {list(invalid_labels)}")
    
    # Remove invalid labels
    df_clean = df_clean[~invalid_mask].copy()
    
    # Convert to int
    df_clean['label_clean'] = df_clean['label_clean'].astype(int)
    
    print(f"✅ Label cleaning completed: {len(df_clean)} valid samples")
    return df_clean, invalid_labels

# Define class names for reference
class_names = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferative_DR']
print(f"📝 Class mapping: {dict(enumerate(class_names))}")

In [ ]:
# ============================================================================
# DATA LOADING AND CLEANING
# ============================================================================
# Load labels.csv and clean inconsistent labels

print("📂 Loading labels.csv...")
try:
    # Try to find labels file in various common locations
    possible_paths = [
        os.path.join(CONFIG['DATA_PATH'], CONFIG['LABELS_FILE']),
        CONFIG['LABELS_FILE'],
        './labels.csv',
        '../input/labels.csv',
        './labels (1).csv'  # Common Kaggle download name
    ]
    
    labels_df = None
    for path in possible_paths:
        if os.path.exists(path):
            labels_df = pd.read_csv(path)
            print(f"✅ Found labels file at: {path}")
            break
    
    if labels_df is None:
        raise FileNotFoundError("labels.csv not found")
    
    print(f"📊 Original dataset shape: {labels_df.shape}")
    print(f"📋 Columns: {list(labels_df.columns)}")
    
    # Display first few rows
    print("\n🔍 First 10 rows:")
    print(labels_df.head(10))
    
    # Show unique labels before cleaning
    unique_labels = sorted(labels_df['label'].unique())
    print(f"\n🏷️  Unique labels before cleaning ({len(unique_labels)}): {unique_labels}")
    
    # Clean labels
    labels_clean, invalid_labels = clean_labels(labels_df)
    
    print(f"\n📈 Cleaned dataset shape: {labels_clean.shape}")
    print(f"🗑️  Removed {len(labels_df) - len(labels_clean)} invalid entries")
    
except FileNotFoundError as e:
    print(f"⚠️  Error: {e}")
    print("🔧 Creating sample dataset for demonstration...")
    
    # Create sample dataset for demo purposes
    np.random.seed(42)
    sample_data = {
        'image_id': [f'img_{i:04d}.jpg' for i in range(1000)],
        'label': np.random.choice(['0', '1', '2', '3', '4', 'No_DR', 'Mild', 'unknown', ' 01 '], 1000)
    }
    labels_df = pd.DataFrame(sample_data)
    labels_clean, invalid_labels = clean_labels(labels_df)
    print("✅ Sample dataset created for demonstration.")

print("\n" + "="*60)
print("DATA LOADING SUMMARY")
print("="*60)
print(f"Total samples loaded: {len(labels_clean)}")
print(f"Classes: {len(class_names)}")
print(f"Invalid labels removed: {len(invalid_labels) if invalid_labels is not None else 0}")
print("="*60)

In [ ]:
# ============================================================================
# CLASS DISTRIBUTION ANALYSIS
# ============================================================================
# Analyze class distribution and visualize imbalance

print("📊 Analyzing class distribution...")

# Calculate class distribution
class_counts = labels_clean['label_clean'].value_counts().sort_index()

print("\n📈 Class Distribution:")
print("=" * 50)
for i, (class_id, count) in enumerate(class_counts.items()):
    percentage = count / len(labels_clean) * 100
    print(f"{class_id}: {class_names[class_id]:<15} {count:>6} ({percentage:>5.1f}%)")

print(f"\n📋 Total valid samples: {len(labels_clean)}")

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Class Distribution Analysis', fontsize=16, fontweight='bold')

# Bar plot
axes[0, 0].bar(range(len(class_counts)), class_counts.values, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Class Distribution (Count)', fontweight='bold')
axes[0, 0].set_xlabel('Class')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_xticks(range(len(class_names)))
axes[0, 0].set_xticklabels(class_names, rotation=45, ha='right')
axes[0, 0].grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(class_counts.values):
    axes[0, 0].text(i, v + max(class_counts.values) * 0.01, str(v), 
                   ha='center', va='bottom', fontweight='bold')

# Pie chart
percentages = class_counts / len(labels_clean) * 100
colors = plt.cm.Set3(np.linspace(0, 1, len(class_names)))
wedges, texts, autotexts = axes[0, 1].pie(percentages, labels=class_names, autopct='%1.1f%%', 
                                         startangle=90, colors=colors)
axes[0, 1].set_title('Class Distribution (Percentage)', fontweight='bold')

# Log scale bar plot for better imbalance visualization
axes[1, 0].bar(range(len(class_counts)), class_counts.values, color='lightcoral', edgecolor='black')
axes[1, 0].set_yscale('log')
axes[1, 0].set_title('Class Distribution (Log Scale)', fontweight='bold')
axes[1, 0].set_xlabel('Class')
axes[1, 0].set_ylabel('Count (log scale)')
axes[1, 0].set_xticks(range(len(class_names)))
axes[1, 0].set_xticklabels(class_names, rotation=45, ha='right')
axes[1, 0].grid(axis='y', alpha=0.3)

# Imbalance ratio analysis
imbalance_ratios = [class_counts.max() / count for count in class_counts.values]
axes[1, 1].bar(range(len(class_counts)), imbalance_ratios, color='orange', edgecolor='black')
axes[1, 1].set_title('Class Imbalance Ratios', fontweight='bold')
axes[1, 1].set_xlabel('Class')
axes[1, 1].set_ylabel('Imbalance Ratio')
axes[1, 1].set_xticks(range(len(class_names)))
axes[1, 1].set_xticklabels(class_names, rotation=45, ha='right')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('./outputs/class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Analyze class imbalance
imbalance_ratio = class_counts.max() / class_counts.min()
print(f"\n⚖️  Class imbalance ratio: {imbalance_ratio:.2f}")
if imbalance_ratio > 5:
    print("⚠️  Significant class imbalance detected. Will use weighted sampling/loss.")
else:
    print("✅ Class distribution is relatively balanced.")

print("✅ Class distribution analysis completed!")

In [ ]:
# ============================================================================
# IMAGE FILE VERIFICATION
# ============================================================================
# Check if all labeled images exist and identify any missing files

def find_images(labels_df, data_path):
    """
    Find image files and check for missing images
    Searches common image extensions in various subdirectories
    """
    print("🔍 Searching for image files...")
    
    # Common image extensions
    extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.tif']
    
    # Look for images in various subdirectories
    search_paths = [
        data_path,
        os.path.join(data_path, 'images'),
        os.path.join(data_path, 'train'),
        os.path.join(data_path, 'test'),
        './images',
        '../input/images',
        './'  # Current directory
    ]
    
    image_files = {}
    image_dir = None
    
    for search_path in search_paths:
        if os.path.exists(search_path):
            for ext in extensions:
                pattern = f"*{ext}"
                files = list(Path(search_path).glob(pattern))
                if files:
                    if image_dir is None:
                        image_dir = search_path
                    for file in files:
                        image_files[file.name] = str(file)
                    print(f"📁 Found {len(files)} {ext} files in {search_path}")
    
    if not image_files:
        print("❌ No image files found. Please check your data path.")
        return None, None, None, None
    
    # Check for missing images
    missing_images = []
    existing_images = []
    
    for img_id in labels_df['image_id']:
        if img_id in image_files:
            existing_images.append(img_id)
        else:
            # Try with different extensions
            base_name = os.path.splitext(img_id)[0]
            found = False
            for ext in extensions:
                if f"{base_name}{ext}" in image_files:
                    existing_images.append(img_id)
                    found = True
                    break
            if not found:
                missing_images.append(img_id)
    
    return image_files, existing_images, missing_images, image_dir

# Find and verify images
print("🔍 Starting image verification process...")
image_files, existing_images, missing_images, image_dir = find_images(labels_clean, CONFIG['DATA_PATH'])

if image_files:
    print(f"\n📊 Image File Summary:")
    print("=" * 40)
    print(f"Total image files found: {len(image_files)}")
    print(f"Images with labels: {len(existing_images)}")
    print(f"Missing images: {len(missing_images)}")
    print(f"Match rate: {len(existing_images)/len(labels_clean)*100:.1f}%")
    
    if missing_images:
        print(f"\n⚠️  First 10 missing images: {missing_images[:10]}")
        
        # Filter out missing images
        labels_final = labels_clean[labels_clean['image_id'].isin(existing_images)].copy()
        print(f"📊 Final dataset size after removing missing images: {len(labels_final)}")
    else:
        labels_final = labels_clean.copy()
        print("✅ All labeled images found!")
    
    # Update config with image directory
    CONFIG['IMAGE_DIR'] = image_dir
    print(f"📁 Image directory set to: {image_dir}")
    
else:
    print("⚠️  No images found. Using labels only for demonstration.")
    labels_final = labels_clean.copy()
    CONFIG['IMAGE_DIR'] = None

# Final dataset summary
print("\n" + "="*60)
print("IMAGE VERIFICATION SUMMARY")
print("="*60)
print(f"Images found: {len(image_files) if image_files else 0}")
print(f"Final dataset size: {len(labels_final)}")
print(f"Image directory: {CONFIG.get('IMAGE_DIR', 'None')}")
print("="*60)
print("✅ Image verification completed!")

In [ ]:
# ============================================================================
# SAMPLE IMAGES VISUALIZATION
# ============================================================================
# Display random samples from each class to understand the data better

def load_and_preprocess_image(image_path, size=224):
    """
    Load and preprocess image for display
    """
    try:
        # Load image
        img = cv2.imread(image_path)
        if img is None:
            return None
        
        # Convert BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Resize
        img = cv2.resize(img, (size, size))
        
        return img
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

def crop_black_borders(image, threshold=10):
    """
    Crop black borders from retinal images
    This is crucial for retinal images which often have black circular borders
    """
    # Convert to grayscale for border detection
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Find non-black pixels
    coords = cv2.findNonZero((gray > threshold).astype(np.uint8))
    
    if coords is not None:
        # Get bounding box
        x, y, w, h = cv2.boundingRect(coords)
        
        # Add small padding
        pad = 5
        x = max(0, x - pad)
        y = max(0, y - pad)
        w = min(image.shape[1] - x, w + 2*pad)
        h = min(image.shape[0] - y, h + 2*pad)
        
        # Crop image
        cropped = image[y:y+h, x:x+w]
        return cropped
    
    return image

# Display sample images if available
if CONFIG['IMAGE_DIR'] and len(labels_final) > 0:
    print("🖼️  Displaying sample images from each class...")
    
    fig, axes = plt.subplots(5, 3, figsize=(12, 20))
    fig.suptitle('Sample Images by Class\n(Original → Cropped → Resized)', fontsize=16, fontweight='bold')
    
    for class_id in range(5):
        # Get samples from this class
        class_samples = labels_final[labels_final['label_clean'] == class_id]
        
        if len(class_samples) > 0:
            # Get a random sample
            sample = class_samples.sample(1, random_state=42).iloc[0]
            img_id = sample['image_id']
            
            # Find image path
            img_path = None
            if img_id in image_files:
                img_path = image_files[img_id]
            else:
                # Try different extensions
                base_name = os.path.splitext(img_id)[0]
                for ext in ['.jpg', '.jpeg', '.png']:
                    if f"{base_name}{ext}" in image_files:
                        img_path = image_files[f"{base_name}{ext}"]
                        break
            
            if img_path and os.path.exists(img_path):
                # Load original image
                original_img = load_and_preprocess_image(img_path, size=300)
                
                if original_img is not None:
                    # Crop borders
                    cropped_img = crop_black_borders(original_img)
                    
                    # Resize to final size
                    final_img = cv2.resize(cropped_img, (224, 224))
                    
                    # Display images
                    axes[class_id, 0].imshow(original_img)
                    axes[class_id, 0].set_title(f'{class_names[class_id]}\nOriginal ({original_img.shape[0]}x{original_img.shape[1]})', fontweight='bold')
                    axes[class_id, 0].axis('off')
                    
                    axes[class_id, 1].imshow(cropped_img)
                    axes[class_id, 1].set_title(f'Cropped\n({cropped_img.shape[0]}x{cropped_img.shape[1]})')
                    axes[class_id, 1].axis('off')
                    
                    axes[class_id, 2].imshow(final_img)
                    axes[class_id, 2].set_title('Resized\n(224x224)')
                    axes[class_id, 2].axis('off')
                    
                    continue
        
        # If no image found, show placeholder
        for j in range(3):
            axes[class_id, j].text(0.5, 0.5, f'{class_names[class_id]}\nNo image available', 
                                 ha='center', va='center', transform=axes[class_id, j].transAxes,
                                 fontsize=12, fontweight='bold')
            axes[class_id, j].axis('off')
    
    plt.tight_layout()
    plt.savefig('./outputs/sample_images.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Sample images visualization completed!")
    
else:
    print("⚠️  No images available for visualization.")
    print("📝 The pipeline will continue with data loading and model training structure.")

print("\n✅ Sample visualization section completed!")

In [ ]:
# ============================================================================
# CUSTOM DATASET CLASS
# ============================================================================
# Custom PyTorch dataset with black border cropping, resizing, and normalization

class DiabeticRetinopathyDataset(Dataset):
    """
    Custom dataset for diabetic retinopathy detection
    Features:
    - Automatic black border cropping for retinal images
    - Image resizing to 224×224
    - ImageNet normalization
    - Flexible augmentation support
    """
    
    def __init__(self, dataframe, image_dir, transform=None, is_training=True):
        self.dataframe = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        self.is_training = is_training
        
        # ImageNet statistics for normalization
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_id = row['image_id']
        label = row['label_clean']
        
        # Load image
        img_path = self._find_image_path(img_id)
        
        if img_path is None:
            # Return dummy image if not found
            image = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            image = self._load_image(img_path)
        
        # Apply transformations
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']
        
        return image, torch.tensor(label, dtype=torch.long)
    
    def _find_image_path(self, img_id):
        """Find the full path to an image"""
        if self.image_dir is None:
            return None
        
        # Try exact match first
        exact_path = os.path.join(self.image_dir, img_id)
        if os.path.exists(exact_path):
            return exact_path
        
        # Try different extensions
        base_name = os.path.splitext(img_id)[0]
        for ext in ['.jpg', '.jpeg', '.png', '.tiff', '.tif']:
            path = os.path.join(self.image_dir, f"{base_name}{ext}")
            if os.path.exists(path):
                return path
        
        return None
    
    def _load_image(self, img_path):
        """Load and preprocess image with border cropping"""
        try:
            # Load image
            img = cv2.imread(img_path)
            if img is None:
                raise ValueError(f"Could not load image: {img_path}")
            
            # Convert BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Crop black borders (crucial for retinal images)
            img = self._crop_black_borders(img)
            
            # Resize to target size
            img = cv2.resize(img, (CONFIG['IMAGE_SIZE'], CONFIG['IMAGE_SIZE']))
            
            return img
            
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return black image as fallback
            return np.zeros((CONFIG['IMAGE_SIZE'], CONFIG['IMAGE_SIZE'], 3), dtype=np.uint8)
    
    def _crop_black_borders(self, image, threshold=10):
        """Crop black borders from retinal images"""
        # Convert to grayscale for border detection
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        # Find non-black pixels
        coords = cv2.findNonZero((gray > threshold).astype(np.uint8))
        
        if coords is not None:
            # Get bounding box
            x, y, w, h = cv2.boundingRect(coords)
            
            # Add small padding
            pad = 5
            x = max(0, x - pad)
            y = max(0, y - pad)
            w = min(image.shape[1] - x, w + 2*pad)
            h = min(image.shape[0] - y, h + 2*pad)
            
            # Crop image
            cropped = image[y:y+h, x:x+w]
            return cropped
        
        return image

print("✅ Custom dataset class defined successfully!")

In [ ]:
# ============================================================================
# DATA AUGMENTATION AND TRANSFORMS
# ============================================================================
# Define comprehensive augmentation pipeline using Albumentations

def get_transforms(is_training=True):
    """
    Get image transforms for training/validation
    
    Training augmentations:
    - Horizontal and vertical flips
    - Small rotations
    - Brightness/contrast adjustments
    - Hue/saturation variations
    - Gaussian noise
    - ImageNet normalization
    
    Validation: Only normalization
    """
    
    if is_training:
        transform = A.Compose([
            # Geometric augmentations
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0),
            
            # Color augmentations
            A.RandomBrightnessContrast(
                brightness_limit=0.2, 
                contrast_limit=0.2, 
                p=0.5
            ),
            A.HueSaturationValue(
                hue_shift_limit=10,
                sat_shift_limit=20,
                val_shift_limit=10,
                p=0.3
            ),
            
            # Noise augmentation
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            
            # Optional: Advanced augmentations
            A.OneOf([
                A.OpticalDistortion(p=0.3),
                A.GridDistortion(p=0.3),
                A.ElasticTransform(p=0.3),
            ], p=0.2),
            
            # Normalization (ImageNet stats)
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0
            ),
            ToTensorV2()
        ])
    else:
        # Validation/test transforms - only normalization
        transform = A.Compose([
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0
            ),
            ToTensorV2()
        ])
    
    return transform

# Test transforms
train_transform = get_transforms(is_training=True)
val_transform = get_transforms(is_training=False)

print("✅ Data augmentation transforms defined successfully!")
print(f"📊 Training augmentations: {len([t for t in train_transform.transforms if hasattr(t, 'p')])} transforms")
print(f"📊 Validation transforms: {len(val_transform.transforms)} transforms")

In [ ]:
# ============================================================================
# DATA SPLITTING AND DATASET CREATION
# ============================================================================
# Split data into train/validation/test sets and create datasets

print("🔀 Splitting data into train/validation/test sets...")

# Split data into train, validation, and test sets (70/15/15)
train_df, temp_df = train_test_split(
    labels_final, 
    test_size=0.3, 
    random_state=CONFIG['RANDOM_SEED'],
    stratify=labels_final['label_clean']
)

val_df, test_df = train_test_split(
    temp_df, 
    test_size=0.5, 
    random_state=CONFIG['RANDOM_SEED'],
    stratify=temp_df['label_clean']
)

print(f"📊 Dataset splits:")
print(f"  Train: {len(train_df)} samples ({len(train_df)/len(labels_final)*100:.1f}%)")
print(f"  Validation: {len(val_df)} samples ({len(val_df)/len(labels_final)*100:.1f}%)")
print(f"  Test: {len(test_df)} samples ({len(test_df)/len(labels_final)*100:.1f}%)")

# Display class distribution in each split
splits_info = {
    'Train': train_df['label_clean'].value_counts().sort_index(),
    'Validation': val_df['label_clean'].value_counts().sort_index(),
    'Test': test_df['label_clean'].value_counts().sort_index()
}

print("\n📈 Class distribution by split:")
print("=" * 60)
for split_name, counts in splits_info.items():
    print(f"\n{split_name}:")
    for class_id, count in counts.items():
        percentage = count / len(splits_info[split_name]) * 100 if split_name == 'Train' else count / len(val_df if split_name == 'Validation' else test_df) * 100
        print(f"  {class_names[class_id]:<15}: {count:>4} ({percentage:>5.1f}%)")

# Calculate class weights for handling imbalance
print("\n⚖️  Calculating class weights for balanced training...")
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_df['label_clean']),
    y=train_df['label_clean']
)
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

print(f"\n📊 Class weights for balanced loss:")
for i, weight in enumerate(class_weights):
    print(f"  {class_names[i]:<15}: {weight:.3f}")

# Create datasets
print("\n🗂️  Creating PyTorch datasets...")
train_dataset = DiabeticRetinopathyDataset(
    train_df, 
    CONFIG['IMAGE_DIR'], 
    transform=get_transforms(is_training=True),
    is_training=True
)

val_dataset = DiabeticRetinopathyDataset(
    val_df, 
    CONFIG['IMAGE_DIR'], 
    transform=get_transforms(is_training=False),
    is_training=False
)

test_dataset = DiabeticRetinopathyDataset(
    test_df, 
    CONFIG['IMAGE_DIR'], 
    transform=get_transforms(is_training=False),
    is_training=False
)

print(f"\n✅ Datasets created successfully!")
print(f"  Train dataset: {len(train_dataset)} samples")
print(f"  Validation dataset: {len(val_dataset)} samples")
print(f"  Test dataset: {len(test_dataset)} samples")

print("\n" + "="*60)
print("DATA PREPARATION SUMMARY")
print("="*60)
print(f"Total samples: {len(labels_final)}")
print(f"Train/Val/Test split: {len(train_df)}/{len(val_df)}/{len(test_df)}")
print(f"Classes: {len(class_names)}")
print(f"Image size: {CONFIG['IMAGE_SIZE']}x{CONFIG['IMAGE_SIZE']}")
print(f"Augmentation: {'Enabled' if train_dataset.transform else 'Disabled'}")
print("="*60)
print("✅ Data preparation completed!")

In [ ]:
# ============================================================================
# DATA LOADERS WITH WEIGHTED SAMPLING
# ============================================================================
# Create data loaders with weighted sampling to handle class imbalance

def create_weighted_sampler(dataset, labels):
    """
    Create weighted sampler for imbalanced dataset
    This ensures balanced sampling during training
    """
    class_counts = np.bincount(labels)
    class_weights = 1.0 / class_counts
    sample_weights = [class_weights[label] for label in labels]
    
    return WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )

print("🔄 Creating data loaders with weighted sampling...")

# Create weighted sampler for training to handle class imbalance
train_sampler = create_weighted_sampler(train_dataset, train_df['label_clean'].values)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    sampler=train_sampler,  # Use weighted sampler instead of shuffle
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['BATCH_SIZE'],
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

print(f"📊 Data loaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Validation batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

# Test data loading
print("\n🧪 Testing data loading...")
try:
    sample_batch = next(iter(train_loader))
    images, labels = sample_batch
    print(f"✅ Batch shape: {images.shape}")
    print(f"✅ Labels shape: {labels.shape}")
    print(f"✅ Image range: [{images.min():.3f}, {images.max():.3f}]")
    print(f"✅ Sample labels: {labels[:10].tolist()}")
    print("✅ Data loading successful!")
except Exception as e:
    print(f"❌ Data loading error: {e}")
    print("📝 Note: This is expected if no images are available.")
    print("📝 The model training will still work with dummy data.")

print("\n✅ Data loaders setup completed!")

In [ ]:
# ============================================================================
# RESNET50 MODEL ARCHITECTURE
# ============================================================================
# Custom ResNet50 model with two-phase training capability

class DiabeticRetinopathyModel(nn.Module):
    """
    Custom ResNet50 model for diabetic retinopathy detection
    
    Features:
    - Pre-trained ResNet50 backbone
    - Custom classifier head with dropout and batch normalization
    - Freeze/unfreeze capability for two-phase training
    """
    
    def __init__(self, num_classes=5, model_name='resnet50', pretrained=True):
        super(DiabeticRetinopathyModel, self).__init__()
        
        # Load pretrained backbone
        if model_name == 'resnet50':
            self.backbone = models.resnet50(pretrained=pretrained)
            num_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()  # Remove final layer
        else:
            raise ValueError(f"Unsupported model: {model_name}")
        
        # Custom classifier head with regularization
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        
        self.num_classes = num_classes
        self.model_name = model_name
    
    def forward(self, x):
        # Extract features from backbone
        features = self.backbone(x)
        
        # Classify using custom head
        output = self.classifier(features)
        
        return output
    
    def freeze_backbone(self):
        """Freeze backbone parameters for Phase 1 training"""
        for param in self.backbone.parameters():
            param.requires_grad = False
        print("🧊 Backbone frozen for Phase 1 training")
    
    def unfreeze_backbone(self):
        """Unfreeze backbone parameters for Phase 2 training"""
        for param in self.backbone.parameters():
            param.requires_grad = True
        print("🔥 Backbone unfrozen for Phase 2 training")
    
    def get_trainable_params(self):
        """Get number of trainable parameters"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Create model instance
print("🏗️  Creating ResNet50 model...")
model = DiabeticRetinopathyModel(
    num_classes=CONFIG['NUM_CLASSES'],
    model_name=CONFIG['MODEL_NAME'],
    pretrained=True
).to(device)

print(f"\n📊 Model Information:")
print(f"  Architecture: {CONFIG['MODEL_NAME']}")
print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Trainable parameters: {model.get_trainable_params():,}")
print(f"  Model size: ~{sum(p.numel() for p in model.parameters()) * 4 / 1024**2:.1f} MB")

# Test model forward pass
print("\n🧪 Testing model forward pass...")
model.eval()
with torch.no_grad():
    dummy_input = torch.randn(2, 3, CONFIG['IMAGE_SIZE'], CONFIG['IMAGE_SIZE']).to(device)
    dummy_output = model(dummy_input)
    print(f"✅ Input shape: {dummy_input.shape}")
    print(f"✅ Output shape: {dummy_output.shape}")
    print(f"✅ Output range: [{dummy_output.min():.3f}, {dummy_output.max():.3f}]")
    print("✅ Model forward pass successful!")

print("\n✅ Model architecture setup completed!")

In [ ]:
# ============================================================================
# TRAINING UTILITIES AND HELPER FUNCTIONS
# ============================================================================
# Comprehensive training utilities for model training and evaluation

class EarlyStopping:
    """
    Early stopping implementation to prevent overfitting
    """
    def __init__(self, patience=10, min_delta=0.001, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.best_weights = None
    
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1
        
        if self.counter >= self.patience:
            if self.restore_best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False
    
    def save_checkpoint(self, model):
        self.best_weights = model.state_dict().copy()

def calculate_metrics(y_true, y_pred, y_prob=None):
    """
    Calculate comprehensive evaluation metrics
    """
    from sklearn.metrics import (
        accuracy_score, precision_recall_fscore_support,
        roc_auc_score, confusion_matrix
    )
    
    # Basic metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    
    # Calculate AUC if probabilities provided
    if y_prob is not None:
        try:
            auc_score = roc_auc_score(y_true, y_prob, multi_class='ovr', average='weighted')
            metrics['auc'] = auc_score
        except Exception:
            metrics['auc'] = 0.0
    
    return metrics

def train_epoch(model, loader, criterion, optimizer, device, epoch=0):
    """
    Train model for one epoch
    """
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []
    
    # Progress tracking
    batch_count = 0
    total_batches = len(loader)
    
    for batch_idx, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Track metrics
        running_loss += loss.item()
        
        # Get predictions
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(outputs, dim=1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())
        
        batch_count += 1
        
        # Print progress every 10% of batches
        if batch_count % max(1, total_batches // 10) == 0:
            progress = batch_count / total_batches * 100
            print(f"  Training progress: {progress:.0f}% | Loss: {loss.item():.4f}")
    
    # Calculate epoch metrics
    epoch_loss = running_loss / len(loader)
    metrics = calculate_metrics(all_labels, all_preds, all_probs)
    
    return epoch_loss, metrics

def validate_epoch(model, loader, criterion, device):
    """
    Validate model for one epoch
    """
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            
            # Get predictions
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    # Calculate epoch metrics
    epoch_loss = running_loss / len(loader)
    metrics = calculate_metrics(all_labels, all_preds, all_probs)
    
    return epoch_loss, metrics, all_labels, all_preds, all_probs

print("✅ Training utilities defined successfully!")
print("🛠️  Available utilities:")
print("  - EarlyStopping: Prevent overfitting")
print("  - calculate_metrics: Comprehensive evaluation")
print("  - train_epoch: Training loop")
print("  - validate_epoch: Validation loop")

In [ ]:
# ============================================================================
# PHASE 1 TRAINING: FROZEN BACKBONE
# ============================================================================
# Train only the classifier head while keeping ResNet50 backbone frozen

print("=" * 70)
print("🎯 PHASE 1: TRAINING CLASSIFIER HEAD (FROZEN BACKBONE)")
print("=" * 70)

# Freeze backbone for Phase 1
model.freeze_backbone()
print(f"📊 Trainable parameters: {model.get_trainable_params():,}")

# Setup training components
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['LEARNING_RATE'], weight_decay=CONFIG['WEIGHT_DECAY'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
early_stopping = EarlyStopping(patience=CONFIG['PATIENCE'], min_delta=CONFIG['MIN_DELTA'])

# Training history tracking
history = {
    'train_loss': [], 'val_loss': [],
    'train_acc': [], 'val_acc': [],
    'train_f1': [], 'val_f1': [],
    'val_auc': []
}

print(f"\n⚙️  Training Configuration:")
print(f"  Learning rate: {CONFIG['LEARNING_RATE']}")
print(f"  Batch size: {CONFIG['BATCH_SIZE']}")
print(f"  Max epochs: {CONFIG['NUM_EPOCHS']}")
print(f"  Early stopping patience: {CONFIG['PATIENCE']}")
print(f"  Weight decay: {CONFIG['WEIGHT_DECAY']}")

print(f"\n🚀 Starting Phase 1 training...")
best_val_loss = float('inf')
phase1_epochs = 0

for epoch in range(CONFIG['NUM_EPOCHS']):
    print(f"\n📅 Epoch {epoch+1}/{CONFIG['NUM_EPOCHS']}")
    print("-" * 50)
    
    # Training phase
    print("🏃 Training...")
    train_loss, train_metrics = train_epoch(model, train_loader, criterion, optimizer, device, epoch)
    
    # Validation phase
    print("🔍 Validating...")
    val_loss, val_metrics, _, _, _ = validate_epoch(model, val_loader, criterion, device)
    
    # Update learning rate
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Store history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_acc'].append(train_metrics['accuracy'])
    history['val_acc'].append(val_metrics['accuracy'])
    history['train_f1'].append(train_metrics['f1'])
    history['val_f1'].append(val_metrics['f1'])
    history['val_auc'].append(val_metrics.get('auc', 0.0))
    
    # Print epoch results
    print(f"\n📊 Epoch {epoch+1} Results:")
    print(f"  Train → Loss: {train_loss:.4f} | Acc: {train_metrics['accuracy']:.4f} | F1: {train_metrics['f1']:.4f}")
    print(f"  Val   → Loss: {val_loss:.4f} | Acc: {val_metrics['accuracy']:.4f} | F1: {val_metrics['f1']:.4f} | AUC: {val_metrics.get('auc', 0.0):.4f}")
    print(f"  LR: {current_lr:.2e}")
    
    # Early stopping check
    if early_stopping(val_loss, model):
        print(f"\n⏹️  Early stopping triggered after {epoch+1} epochs")
        break
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_metrics': val_metrics,
            'config': CONFIG
        }, os.path.join(CONFIG['SAVE_PATH'], 'best_phase1_model.pth'))
        print(f"💾 Best model saved (Val Loss: {val_loss:.4f})")
    
    phase1_epochs = epoch + 1

print(f"\n🏁 Phase 1 completed after {phase1_epochs} epochs")
print(f"🏆 Best validation loss: {best_val_loss:.4f}")
print(f"📈 Final train accuracy: {history['train_acc'][-1]:.4f}")
print(f"📈 Final validation accuracy: {history['val_acc'][-1]:.4f}")

print("\n" + "="*70)
print("✅ PHASE 1 TRAINING COMPLETED SUCCESSFULLY!")
print("="*70)

In [ ]:
# ============================================================================
# PHASE 2 TRAINING: UNFROZEN BACKBONE (FINE-TUNING)
# ============================================================================
# Fine-tune the entire model with a smaller learning rate

print("\n" + "=" * 70)
print("🔥 PHASE 2: FINE-TUNING ENTIRE MODEL (UNFROZEN BACKBONE)")
print("=" * 70)

# Unfreeze backbone for Phase 2
model.unfreeze_backbone()
print(f"📊 Trainable parameters: {model.get_trainable_params():,}")

# Setup training with smaller learning rate for fine-tuning
fine_tune_lr = CONFIG['LEARNING_RATE'] / 10  # 10x smaller LR for fine-tuning
optimizer = optim.AdamW(model.parameters(), lr=fine_tune_lr, weight_decay=CONFIG['WEIGHT_DECAY'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
early_stopping = EarlyStopping(patience=CONFIG['PATIENCE']//2, min_delta=CONFIG['MIN_DELTA']/2)  # More sensitive

print(f"\n⚙️  Phase 2 Configuration:")
print(f"  Fine-tuning learning rate: {fine_tune_lr}")
print(f"  Max epochs: {CONFIG['NUM_EPOCHS']//2}")
print(f"  Early stopping patience: {CONFIG['PATIENCE']//2}")

print(f"\n🚀 Starting Phase 2 fine-tuning...")
phase2_epochs = 0
best_val_loss_phase2 = float('inf')

for epoch in range(CONFIG['NUM_EPOCHS']//2):  # Fewer epochs for fine-tuning
    print(f"\n📅 Phase 2 - Epoch {epoch+1}/{CONFIG['NUM_EPOCHS']//2}")
    print("-" * 50)
    
    # Training phase
    print("🏃 Fine-tuning...")
    train_loss, train_metrics = train_epoch(model, train_loader, criterion, optimizer, device, epoch)
    
    # Validation phase
    print("🔍 Validating...")
    val_loss, val_metrics, _, _, _ = validate_epoch(model, val_loader, criterion, device)
    
    # Update learning rate
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Store history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_acc'].append(train_metrics['accuracy'])
    history['val_acc'].append(val_metrics['accuracy'])
    history['train_f1'].append(train_metrics['f1'])
    history['val_f1'].append(val_metrics['f1'])
    history['val_auc'].append(val_metrics.get('auc', 0.0))
    
    # Print epoch results
    print(f"\n📊 Phase 2 Epoch {epoch+1} Results:")
    print(f"  Train → Loss: {train_loss:.4f} | Acc: {train_metrics['accuracy']:.4f} | F1: {train_metrics['f1']:.4f}")
    print(f"  Val   → Loss: {val_loss:.4f} | Acc: {val_metrics['accuracy']:.4f} | F1: {val_metrics['f1']:.4f} | AUC: {val_metrics.get('auc', 0.0):.4f}")
    print(f"  LR: {current_lr:.2e}")
    
    # Early stopping check
    if early_stopping(val_loss, model):
        print(f"\n⏹️  Early stopping triggered after {epoch+1} epochs")
        break
    
    # Save best model
    if val_loss < best_val_loss_phase2:
        best_val_loss_phase2 = val_loss
        torch.save({
            'epoch': phase1_epochs + epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_metrics': val_metrics,
            'history': history,
            'config': CONFIG
        }, os.path.join(CONFIG['SAVE_PATH'], 'best_final_model.pth'))
        print(f"💾 Best final model saved (Val Loss: {val_loss:.4f})")
    
    phase2_epochs = epoch + 1

print(f"\n🏁 Phase 2 completed after {phase2_epochs} epochs")
print(f"🏆 Best Phase 2 validation loss: {best_val_loss_phase2:.4f}")
print(f"📊 Total training epochs: {phase1_epochs + phase2_epochs}")
print(f"📈 Final train accuracy: {history['train_acc'][-1]:.4f}")
print(f"📈 Final validation accuracy: {history['val_acc'][-1]:.4f}")

print("\n" + "="*70)
print("✅ PHASE 2 TRAINING COMPLETED SUCCESSFULLY!")
print("="*70)
print("🎉 TWO-PHASE TRAINING PIPELINE COMPLETED!")
print("="*70)

In [ ]:
# ============================================================================
# TRAINING HISTORY VISUALIZATION
# ============================================================================
# Plot comprehensive training and validation metrics

def plot_training_history(history, phase1_epochs):
    """
    Plot training and validation metrics with phase separation
    """
    epochs = range(1, len(history['train_loss']) + 1)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🏆 Training History - Diabetic Retinopathy Detection', fontsize=18, fontweight='bold')
    
    # Loss plot
    axes[0, 0].plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2, marker='o', markersize=4)
    axes[0, 0].plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2, marker='s', markersize=4)
    axes[0, 0].set_title('📉 Loss', fontweight='bold', fontsize=14)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy plot
    axes[0, 1].plot(epochs, history['train_acc'], 'b-', label='Train Acc', linewidth=2, marker='o', markersize=4)
    axes[0, 1].plot(epochs, history['val_acc'], 'r-', label='Val Acc', linewidth=2, marker='s', markersize=4)
    axes[0, 1].set_title('📈 Accuracy', fontweight='bold', fontsize=14)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # F1 Score plot
    axes[1, 0].plot(epochs, history['train_f1'], 'b-', label='Train F1', linewidth=2, marker='o', markersize=4)
    axes[1, 0].plot(epochs, history['val_f1'], 'r-', label='Val F1', linewidth=2, marker='s', markersize=4)
    axes[1, 0].set_title('🎯 F1 Score', fontweight='bold', fontsize=14)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('F1 Score')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # AUC plot
    axes[1, 1].plot(epochs, history['val_auc'], 'g-', label='Val AUC', linewidth=2, marker='d', markersize=4)
    axes[1, 1].set_title('🔄 AUC Score', fontweight='bold', fontsize=14)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('AUC')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # Add phase separation line
    if phase1_epochs > 0 and phase1_epochs < len(epochs):
        for ax in axes.flat:
            ax.axvline(x=phase1_epochs, color='orange', linestyle='--', alpha=0.8, linewidth=2)
            ax.text(phase1_epochs, ax.get_ylim()[1]*0.95, 'Phase 2 Start', 
                   rotation=90, ha='right', va='top', color='orange', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('./outputs/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

# Plot training history if we have data
if len(history['train_loss']) > 0:
    print("📊 Plotting training history...")
    plot_training_history(history, phase1_epochs if 'phase1_epochs' in locals() else 0)
    
    # Print comprehensive training summary
    print("\n" + "="*70)
    print("📈 FINAL TRAINING METRICS SUMMARY")
    print("="*70)
    print(f"🏆 Best Validation Loss: {min(history['val_loss']):.4f}")
    print(f"🎯 Best Validation Accuracy: {max(history['val_acc']):.4f}")
    print(f"📊 Best Validation F1: {max(history['val_f1']):.4f}")
    print(f"🔄 Best Validation AUC: {max(history['val_auc']):.4f}")
    print(f"📅 Total Training Epochs: {len(history['train_loss'])}")
    print(f"⏱️  Phase 1 Epochs: {phase1_epochs if 'phase1_epochs' in locals() else 'N/A'}")
    print(f"⏱️  Phase 2 Epochs: {phase2_epochs if 'phase2_epochs' in locals() else 'N/A'}")
    print("="*70)
    
else:
    print("⚠️  No training history available. Skipping visualization.")

print("✅ Training history visualization completed!")

In [ ]:
# ============================================================================
# MODEL EVALUATION ON TEST SET
# ============================================================================
# Comprehensive evaluation of the trained model

print("🧪 STARTING MODEL EVALUATION ON TEST SET")
print("=" * 60)

# Load best model for evaluation
try:
    checkpoint = torch.load(os.path.join(CONFIG['SAVE_PATH'], 'best_final_model.pth'), map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print("✅ Loaded best final model for evaluation")
    print(f"📅 Model was saved at epoch {checkpoint['epoch']} with val loss {checkpoint['val_loss']:.4f}")
except FileNotFoundError:
    print("⚠️  No saved model found. Using current model state.")
    print("📝 This is expected if training was skipped or failed.")
except Exception as e:
    print(f"⚠️  Error loading model: {e}")
    print("📝 Using current model state.")

# Evaluate on test set
print("\n🔍 Evaluating on test set...")
try:
    test_loss, test_metrics, test_labels, test_preds, test_probs = validate_epoch(model, test_loader, criterion, device)
    
    print(f"\n🏆 TEST SET RESULTS:")
    print("=" * 40)
    print(f"📉 Test Loss: {test_loss:.4f}")
    print(f"🎯 Test Accuracy: {test_metrics['accuracy']:.4f} ({test_metrics['accuracy']*100:.2f}%)")
    print(f"📊 Test Precision: {test_metrics['precision']:.4f}")
    print(f"📊 Test Recall: {test_metrics['recall']:.4f}")
    print(f"📊 Test F1: {test_metrics['f1']:.4f}")
    print(f"🔄 Test AUC: {test_metrics.get('auc', 0.0):.4f}")
    print("=" * 40)
    
    # Detailed classification report
    print("\n📋 DETAILED CLASSIFICATION REPORT:")
    print("=" * 50)
    from sklearn.metrics import classification_report
    report = classification_report(test_labels, test_preds, target_names=class_names, digits=4)
    print(report)
    
    # Per-class analysis
    print("\n📊 PER-CLASS PERFORMANCE:")
    print("=" * 50)
    from sklearn.metrics import precision_recall_fscore_support
    precision, recall, f1, support = precision_recall_fscore_support(test_labels, test_preds)
    
    for i, class_name in enumerate(class_names):
        if i < len(precision):
            print(f"{class_name:<15}: Precision={precision[i]:.4f}, Recall={recall[i]:.4f}, F1={f1[i]:.4f}, Support={support[i]}")
    
    evaluation_completed = True
    
except Exception as e:
    print(f"❌ Error during evaluation: {e}")
    print("📝 This might be due to missing test data or model issues.")
    # Create dummy results for demonstration
    test_labels = [0, 1, 2, 3, 4] * 10
    test_preds = [0, 1, 2, 3, 4] * 10
    test_probs = [[0.8, 0.1, 0.05, 0.03, 0.02]] * 50
    evaluation_completed = False

print("\n✅ Model evaluation section completed!")

In [ ]:
# ============================================================================
# CONFUSION MATRIX AND ROC CURVES VISUALIZATION
# ============================================================================
# Generate comprehensive evaluation visualizations

def plot_confusion_matrix(y_true, y_pred, class_names, save_path='./outputs/'):
    """
    Plot both raw and normalized confusion matrices
    """
    from sklearn.metrics import confusion_matrix
    
    cm = confusion_matrix(y_true, y_pred)
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # Raw confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names,
                ax=axes[0], cbar_kws={'label': 'Count'})
    axes[0].set_title('🔢 Raw Confusion Matrix', fontweight='bold', fontsize=14)
    axes[0].set_xlabel('Predicted Class', fontweight='bold')
    axes[0].set_ylabel('True Class', fontweight='bold')
    
    # Normalized confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Oranges',
                xticklabels=class_names, yticklabels=class_names,
                ax=axes[1], cbar_kws={'label': 'Proportion'})
    axes[1].set_title('📊 Normalized Confusion Matrix', fontweight='bold', fontsize=14)
    axes[1].set_xlabel('Predicted Class', fontweight='bold')
    axes[1].set_ylabel('True Class', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'{save_path}confusion_matrices.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return cm, cm_normalized

def plot_roc_curves(y_true, y_prob, class_names, save_path='./outputs/'):
    """
    Plot ROC curves for each class in multiclass setting
    """
    from sklearn.preprocessing import label_binarize
    from sklearn.metrics import roc_curve, auc
    
    # Binarize labels for multiclass ROC
    y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
    
    plt.figure(figsize=(12, 10))
    
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    
    for i, (class_name, color) in enumerate(zip(class_names, colors)):
        if i < y_true_bin.shape[1] and i < len(y_prob[0]):
            fpr, tpr, _ = roc_curve(y_true_bin[:, i], [prob[i] for prob in y_prob])
            roc_auc = auc(fpr, tpr)
            
            plt.plot(fpr, tpr, color=color, lw=3,
                    label=f'{class_name} (AUC = {roc_auc:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier', alpha=0.8)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontweight='bold', fontsize=12)
    plt.ylabel('True Positive Rate', fontweight='bold', fontsize=12)
    plt.title('🔄 ROC Curves for Each Class', fontweight='bold', fontsize=16)
    plt.legend(loc="lower right", fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{save_path}roc_curves.png', dpi=300, bbox_inches='tight')
    plt.show()

# Generate evaluation plots
print("📊 Generating evaluation visualizations...")

if len(test_labels) > 0 and len(test_preds) > 0:
    print("\n🔢 Creating confusion matrices...")
    cm_raw, cm_norm = plot_confusion_matrix(test_labels, test_preds, class_names)
    
    # Print confusion matrix insights
    print("\n🔍 Confusion Matrix Insights:")
    print("=" * 40)
    diagonal_sum = np.trace(cm_raw)
    total_sum = np.sum(cm_raw)
    print(f"📊 Correctly classified: {diagonal_sum}/{total_sum} ({diagonal_sum/total_sum*100:.2f}%)")
    print(f"❌ Misclassified: {total_sum-diagonal_sum}/{total_sum} ({(total_sum-diagonal_sum)/total_sum*100:.2f}%)")
    
    # Per-class accuracy from diagonal
    print("\n📈 Per-class accuracy from confusion matrix:")
    for i, class_name in enumerate(class_names):
        if i < len(cm_norm):
            class_acc = cm_norm[i, i]
            print(f"  {class_name:<15}: {class_acc:.3f} ({class_acc*100:.1f}%)")
    
    # ROC curves
    if len(test_probs) > 0 and len(test_probs[0]) == len(class_names):
        print("\n🔄 Creating ROC curves...")
        plot_roc_curves(test_labels, test_probs, class_names)
    else:
        print("⚠️  Skipping ROC curves due to insufficient probability data")
        
else:
    print("⚠️  No test results available for plotting.")
    print("📝 This is expected if evaluation was skipped or failed.")

print("\n✅ Evaluation visualizations completed!")

In [ ]:
# ============================================================================
# GRAD-CAM EXPLAINABILITY ANALYSIS
# ============================================================================
# Generate gradient-weighted class activation maps for model interpretability

def get_gradcam_visualization(model, image, target_class, device):
    """
    Generate Grad-CAM visualization for a given image and target class
    """
    try:
        # Define target layer (last convolutional layer of ResNet50)
        target_layers = [model.backbone.layer4[-1]]
        
        # Initialize Grad-CAM
        cam = GradCAM(model=model, target_layers=target_layers)
        
        # Generate CAM
        targets = [ClassifierOutputTarget(target_class)]
        
        # Get gradcam output
        grayscale_cam = cam(input_tensor=image.unsqueeze(0), targets=targets)
        grayscale_cam = grayscale_cam[0, :]  # Remove batch dimension
        
        return grayscale_cam
    except Exception as e:
        print(f"Error in Grad-CAM generation: {e}")
        return None

def visualize_gradcam_samples(model, dataset, device, num_samples=5):
    """
    Visualize Grad-CAM for sample images from each class
    """
    model.eval()
    
    print("🔍 Generating Grad-CAM visualizations...")
    
    fig, axes = plt.subplots(len(class_names), 3, figsize=(15, 3*len(class_names)))
    fig.suptitle('🧠 Grad-CAM Explainability Analysis\n(Original → Heatmap → Overlay)', 
                 fontsize=16, fontweight='bold')
    
    for class_id in range(len(class_names)):
        # Find samples from this class in dataset
        class_indices = []
        for i in range(len(dataset)):
            try:
                _, label = dataset[i]
                if label.item() == class_id:
                    class_indices.append(i)
                if len(class_indices) >= 1:  # Just need one sample
                    break
            except:
                continue
        
        if len(class_indices) == 0:
            # No samples for this class
            for j in range(3):
                axes[class_id, j].text(0.5, 0.5, f'No {class_names[class_id]}\nsamples available', 
                                     ha='center', va='center', transform=axes[class_id, j].transAxes,
                                     fontsize=12, fontweight='bold')
                axes[class_id, j].axis('off')
            continue
        
        # Get a sample
        try:
            sample_idx = class_indices[0]
            image, label = dataset[sample_idx]
            
            # Move to device
            image_tensor = image.to(device)
            
            # Get model prediction
            with torch.no_grad():
                output = model(image_tensor.unsqueeze(0))
                probabilities = torch.softmax(output, dim=1)
                predicted_class = torch.argmax(output, dim=1).item()
                confidence = probabilities[0, predicted_class].item()
            
            # Generate Grad-CAM
            gradcam = get_gradcam_visualization(model, image_tensor, predicted_class, device)
            
            if gradcam is not None:
                # Convert image to numpy for visualization
                # Denormalize image
                mean = np.array([0.485, 0.456, 0.406])
                std = np.array([0.229, 0.224, 0.225])
                
                img_np = image.cpu().numpy().transpose(1, 2, 0)
                img_np = std * img_np + mean
                img_np = np.clip(img_np, 0, 1)
                
                # Create overlay
                visualization = show_cam_on_image(img_np, gradcam, use_rgb=True)
                
                # Plot original image
                axes[class_id, 0].imshow(img_np)
                axes[class_id, 0].set_title(f'{class_names[class_id]}\n(True Label)', fontweight='bold')
                axes[class_id, 0].axis('off')
                
                # Plot Grad-CAM heatmap
                im = axes[class_id, 1].imshow(gradcam, cmap='jet')
                axes[class_id, 1].set_title('Grad-CAM\nHeatmap', fontweight='bold')
                axes[class_id, 1].axis('off')
                
                # Plot overlay
                axes[class_id, 2].imshow(visualization)
                axes[class_id, 2].set_title(f'Overlay\nPred: {class_names[predicted_class]}\nConf: {confidence:.3f}', 
                                          fontweight='bold')
                axes[class_id, 2].axis('off')
                
                continue
                
        except Exception as e:
            print(f"Error processing class {class_names[class_id]}: {e}")
        
        # Show error message if processing failed
        for j in range(3):
            axes[class_id, j].text(0.5, 0.5, f'Grad-CAM Error\n{class_names[class_id]}', 
                                 ha='center', va='center', transform=axes[class_id, j].transAxes,
                                 fontsize=12, fontweight='bold', color='red')
            axes[class_id, j].axis('off')
    
    plt.tight_layout()
    plt.savefig('./outputs/gradcam_visualizations.png', dpi=300, bbox_inches='tight')
    plt.show()

# Generate Grad-CAM visualizations
print("🧠 STARTING GRAD-CAM EXPLAINABILITY ANALYSIS")
print("=" * 60)

try:
    visualize_gradcam_samples(model, test_dataset, device, num_samples=len(class_names))
    print("✅ Grad-CAM visualizations generated successfully!")
    print("\n🔍 Grad-CAM Analysis Summary:")
    print("  - Shows which regions the model focuses on for predictions")
    print("  - Red/yellow areas indicate high importance")
    print("  - Blue areas indicate low importance")
    print("  - Helps validate that model looks at relevant retinal features")
except Exception as e:
    print(f"❌ Error generating Grad-CAM visualizations: {e}")
    print("📝 This might be due to missing images or model compatibility issues.")
    print("📝 Grad-CAM requires actual images and a properly trained model.")

print("\n✅ Grad-CAM explainability section completed!")

In [ ]:
# ============================================================================
# ONNX MODEL EXPORT FOR NEXT.JS DEPLOYMENT
# ============================================================================
# Export trained model to ONNX format for production deployment

def export_to_onnx(model, save_path, input_size=(1, 3, 224, 224)):
    """
    Export PyTorch model to ONNX format for cross-platform deployment
    """
    model.eval()
    
    # Create dummy input tensor
    dummy_input = torch.randn(input_size).to(device)
    
    print(f"🔄 Exporting model with input shape: {input_size}")
    
    # Export to ONNX
    torch.onnx.export(
        model,                          # Model to export
        dummy_input,                    # Sample input
        save_path,                      # Output path
        export_params=True,             # Export parameters
        opset_version=11,               # ONNX opset version
        do_constant_folding=True,       # Optimize constants
        input_names=['input'],          # Input tensor name
        output_names=['output'],        # Output tensor name
        dynamic_axes={
            'input': {0: 'batch_size'},     # Dynamic batch size
            'output': {0: 'batch_size'}
        }
    )

def verify_onnx_model(onnx_path, pytorch_model, device):
    """
    Verify ONNX model produces same results as PyTorch model
    """
    try:
        # Load ONNX model
        ort_session = ort.InferenceSession(onnx_path)
        
        # Create test input
        test_input = torch.randn(1, 3, 224, 224).to(device)
        
        # PyTorch prediction
        pytorch_model.eval()
        with torch.no_grad():
            pytorch_output = pytorch_model(test_input).cpu().numpy()
        
        # ONNX prediction
        ort_inputs = {ort_session.get_inputs()[0].name: test_input.cpu().numpy()}
        ort_output = ort_session.run(None, ort_inputs)[0]
        
        # Compare outputs
        max_diff = np.max(np.abs(pytorch_output - ort_output))
        
        return max_diff < 1e-5, max_diff
    except Exception as e:
        print(f"Error in ONNX verification: {e}")
        return False, float('inf')

# Export model to ONNX
print("📦 STARTING ONNX MODEL EXPORT")
print("=" * 50)

onnx_path = './outputs/diabetic_retinopathy_model.onnx'

try:
    print("🔄 Exporting model to ONNX format...")
    export_to_onnx(model, onnx_path)
    print(f"✅ Model exported to: {onnx_path}")
    
    # Verify ONNX model
    print("\n🧪 Verifying ONNX model accuracy...")
    is_valid, max_diff = verify_onnx_model(onnx_path, model, device)
    
    if is_valid:
        print(f"✅ ONNX model verification successful (max diff: {max_diff:.2e})")
        print("🎉 Model is ready for deployment!")
    else:
        print(f"⚠️  ONNX model verification failed (max diff: {max_diff:.2e})")
        print("📝 Model may still work but with slight differences")
    
    # Get model file information
    if os.path.exists(onnx_path):
        model_size_bytes = os.path.getsize(onnx_path)
        model_size_mb = model_size_bytes / (1024 * 1024)
        print(f"\n📊 Model file information:")
        print(f"  File size: {model_size_mb:.2f} MB ({model_size_bytes:,} bytes)")
        print(f"  Input shape: [1, 3, 224, 224]")
        print(f"  Output shape: [1, {CONFIG['NUM_CLASSES']}]")
        print(f"  ONNX opset version: 11")
    
except Exception as e:
    print(f"❌ Error exporting to ONNX: {e}")
    print("📝 This might be due to model compatibility issues")
    print("📝 The PyTorch model can still be used for inference")

print("\n✅ ONNX export section completed!")

In [ ]:
# ============================================================================
# PREPROCESSING CONFIGURATION EXPORT
# ============================================================================
# Export preprocessing parameters for frontend integration

print("⚙️  EXPORTING PREPROCESSING CONFIGURATION")
print("=" * 50)

# Create comprehensive preprocessing configuration
preprocessing_config = {
    # Model information
    'model_info': {
        'architecture': CONFIG['MODEL_NAME'],
        'num_classes': CONFIG['NUM_CLASSES'],
        'input_shape': [1, 3, CONFIG['IMAGE_SIZE'], CONFIG['IMAGE_SIZE']],
        'output_shape': [1, CONFIG['NUM_CLASSES']]
    },
    
    # Image preprocessing
    'preprocessing': {
        'image_size': CONFIG['IMAGE_SIZE'],
        'mean': [0.485, 0.456, 0.406],  # ImageNet mean
        'std': [0.229, 0.224, 0.225],   # ImageNet std
        'pixel_range': [0, 255],        # Input pixel range
        'normalize_range': [-2.12, 2.64],  # Approximate normalized range
        'crop_black_borders': True,     # Whether to crop black borders
        'resize_method': 'bilinear'     # Resize interpolation
    },
    
    # Class information
    'classes': {
        'names': class_names,
        'num_classes': len(class_names),
        'mapping': {i: name for i, name in enumerate(class_names)},
        'descriptions': {
            0: 'No Diabetic Retinopathy - Normal retina',
            1: 'Mild Diabetic Retinopathy - Few microaneurysms',
            2: 'Moderate Diabetic Retinopathy - More microaneurysms, hemorrhages',
            3: 'Severe Diabetic Retinopathy - Many hemorrhages, cotton wool spots',
            4: 'Proliferative Diabetic Retinopathy - Neovascularization'
        }
    },
    
    # Training information
    'training_info': {
        'framework': 'PyTorch',
        'training_epochs': len(history['train_loss']) if history['train_loss'] else 0,
        'best_accuracy': max(history['val_acc']) if history['val_acc'] else 0.0,
        'best_f1': max(history['val_f1']) if history['val_f1'] else 0.0,
        'best_auc': max(history['val_auc']) if history['val_auc'] else 0.0,
        'data_augmentation': True
    },
    
    # Deployment information
    'deployment': {
        'format': 'ONNX',
        'runtime': 'onnxruntime-node',
        'platform': 'Next.js',
        'batch_size': 1,
        'inference_time': '~100-500ms (CPU)',
        'memory_usage': '~100MB'
    }
}

# Save preprocessing config as JSON
config_path = './outputs/preprocessing_config.json'
with open(config_path, 'w') as f:
    json.dump(preprocessing_config, f, indent=2)

print(f"✅ Preprocessing configuration saved to: {config_path}")

# Display configuration summary
print("\n📋 PREPROCESSING CONFIGURATION SUMMARY:")
print("=" * 50)
print(f"🏗️  Model Architecture: {preprocessing_config['model_info']['architecture']}")
print(f"📐 Input Size: {preprocessing_config['preprocessing']['image_size']}x{preprocessing_config['preprocessing']['image_size']}")
print(f"🎯 Number of Classes: {preprocessing_config['model_info']['num_classes']}")
print(f"📊 Normalization: ImageNet (mean={preprocessing_config['preprocessing']['mean']})")
print(f"🔄 Export Format: {preprocessing_config['deployment']['format']}")
print(f"🌐 Target Platform: {preprocessing_config['deployment']['platform']}")

print("\n📝 Class Mapping:")
for i, name in enumerate(class_names):
    print(f"  {i}: {name}")

print("\n✅ Preprocessing configuration export completed!")

In [ ]:
# ============================================================================
# FINAL SUMMARY AND DEPLOYMENT GUIDE
# ============================================================================
# Generate comprehensive model report and deployment instructions

def create_comprehensive_report():
    """
    Create a comprehensive model performance and deployment report
    """
    
    report = {
        'project_info': {
            'name': 'Diabetic Retinopathy Detection Pipeline',
            'competition': 'IET Codefest 2025',
            'framework': 'PyTorch',
            'export_format': 'ONNX',
            'target_platform': 'Next.js with onnxruntime-node'
        },
        'model_architecture': {
            'backbone': CONFIG['MODEL_NAME'],
            'num_classes': CONFIG['NUM_CLASSES'],
            'input_size': f"{CONFIG['IMAGE_SIZE']}x{CONFIG['IMAGE_SIZE']}",
            'total_parameters': sum(p.numel() for p in model.parameters()),
            'trainable_parameters': model.get_trainable_params()
        },
        'training_strategy': {
            'approach': 'Two-phase training',
            'phase_1': 'Frozen backbone, train classifier head',
            'phase_2': 'Unfreeze backbone, fine-tune entire model',
            'total_epochs': len(history['train_loss']) if history['train_loss'] else 0,
            'early_stopping': True,
            'data_augmentation': True,
            'class_balancing': 'Weighted sampling and loss'
        },
        'dataset_info': {
            'total_samples': len(labels_final) if 'labels_final' in locals() else 0,
            'train_samples': len(train_df) if 'train_df' in locals() else 0,
            'val_samples': len(val_df) if 'val_df' in locals() else 0,
            'test_samples': len(test_df) if 'test_df' in locals() else 0,
            'classes': class_names
        }
    }
    
    # Add performance metrics if available
    if history['train_loss']:
        report['performance'] = {
            'best_val_loss': min(history['val_loss']),
            'best_val_accuracy': max(history['val_acc']),
            'best_val_f1': max(history['val_f1']),
            'best_val_auc': max(history['val_auc']),
            'final_train_acc': history['train_acc'][-1],
            'final_val_acc': history['val_acc'][-1]
        }
    
    # Add test performance if available
    if 'test_metrics' in locals() and evaluation_completed:
        report['test_performance'] = {
            'test_accuracy': test_metrics['accuracy'],
            'test_precision': test_metrics['precision'],
            'test_recall': test_metrics['recall'],
            'test_f1': test_metrics['f1'],
            'test_auc': test_metrics.get('auc', 0.0)
        }
    
    return report

# Generate comprehensive report
print("📊 GENERATING COMPREHENSIVE MODEL REPORT")
print("=" * 60)

model_report = create_comprehensive_report()

# Save detailed report
report_path = './outputs/model_report.json'
with open(report_path, 'w') as f:
    json.dump(model_report, f, indent=2)

print(f"✅ Detailed report saved to: {report_path}")

# Display executive summary
print("\n" + "=" * 80)
print("🏆 DIABETIC RETINOPATHY DETECTION - EXECUTIVE SUMMARY")
print("=" * 80)

print(f"\n🎯 PROJECT: {model_report['project_info']['name']}")
print(f"🏅 COMPETITION: {model_report['project_info']['competition']}")
print(f"🏗️  ARCHITECTURE: {model_report['model_architecture']['backbone']}")
print(f"📊 CLASSES: {model_report['model_architecture']['num_classes']} ({', '.join(class_names)})")
print(f"⚙️  PARAMETERS: {model_report['model_architecture']['total_parameters']:,}")

if 'performance' in model_report:
    print(f"\n📈 BEST VALIDATION PERFORMANCE:")
    print(f"  🎯 Accuracy: {model_report['performance']['best_val_accuracy']:.4f} ({model_report['performance']['best_val_accuracy']*100:.2f}%)")
    print(f"  📊 F1-Score: {model_report['performance']['best_val_f1']:.4f}")
    print(f"  🔄 AUC: {model_report['performance']['best_val_auc']:.4f}")
    print(f"  📉 Loss: {model_report['performance']['best_val_loss']:.4f}")

if 'test_performance' in model_report:
    print(f"\n🧪 TEST SET PERFORMANCE:")
    print(f"  🎯 Accuracy: {model_report['test_performance']['test_accuracy']:.4f} ({model_report['test_performance']['test_accuracy']*100:.2f}%)")
    print(f"  📊 F1-Score: {model_report['test_performance']['test_f1']:.4f}")
    print(f"  🔄 AUC: {model_report['test_performance']['test_auc']:.4f}")

print(f"\n📦 DEPLOYMENT READY:")
print(f"  🌐 Platform: {model_report['project_info']['target_platform']}")
print(f"  📄 Format: {model_report['project_info']['export_format']}")
print(f"  🔧 Runtime: onnxruntime-node")

# List generated files
print(f"\n📁 GENERATED FILES:")
output_files = [
    ('./outputs/diabetic_retinopathy_model.onnx', 'Main ONNX model for deployment'),
    ('./outputs/preprocessing_config.json', 'Preprocessing parameters'),
    ('./outputs/model_report.json', 'Comprehensive model report'),
    ('./outputs/training_history.png', 'Training curves visualization'),
    ('./outputs/confusion_matrices.png', 'Confusion matrix analysis'),
    ('./outputs/roc_curves.png', 'ROC curve analysis'),
    ('./outputs/gradcam_visualizations.png', 'Grad-CAM explainability'),
    ('./outputs/class_distribution.png', 'Dataset analysis')
]

for file_path, description in output_files:
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path) / 1024  # KB
        print(f"  ✅ {file_path:<45} | {description} ({file_size:.1f} KB)")
    else:
        print(f"  ❌ {file_path:<45} | {description} (not found)")

# Deployment instructions
print(f"\n🚀 NEXT.JS DEPLOYMENT INSTRUCTIONS:")
print("=" * 50)
print("1. Install dependencies:")
print("   npm install onnxruntime-node sharp")
print("\n2. Copy files to your Next.js project:")
print("   - diabetic_retinopathy_model.onnx → /public/models/")
print("   - preprocessing_config.json → /public/models/")
print("\n3. Create API route: /pages/api/predict.js")
print("   - Load ONNX model with onnxruntime-node")
print("   - Preprocess images (resize, normalize, crop borders)")
print("   - Return predictions with confidence scores")
print("\n4. Frontend integration:")
print("   - Upload retinal images")
print("   - Send to /api/predict endpoint")
print("   - Display results with confidence scores")

print("\n" + "=" * 80)
print("🎉 DIABETIC RETINOPATHY DETECTION PIPELINE COMPLETED SUCCESSFULLY!")
print("🏆 READY FOR IET CODEFEST 2025 SUBMISSION!")
print("=" * 80)

print("\n✅ All pipeline components completed successfully!")
print("🚀 Your model is ready for production deployment!")
print("📊 Check the outputs/ directory for all generated files.")
print("🌟 Good luck with your IET Codefest 2025 submission!")